# Generación de Respuestas Médicas con un Modelo de Lenguaje

En este notebook, vamos a desarrollar un modelo de lenguaje que pueda generar respuestas médicas coherentes y relevantes a partir de preguntas o descripciones de síntomas proporcionadas por los usuarios. Utilizaremos técnicas de aprendizaje profundo y procesamiento del lenguaje natural para entrenar el modelo con un conjunto de datos de conversaciones médicas.


In [6]:
import torch # type: ignore
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import Dataset, DataLoader # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore


## Preprocesamiento de los Datos

Antes de entrenar el modelo, necesitamos realizar algunos pasos de preprocesamiento en los datos:

1. Cargar el conjunto de datos de conversaciones médicas desde un archivo CSV.
2. Tokenizar las preguntas y respuestas utilizando un tokenizador adecuado.
3. Crear un dataset personalizado para manejar los pares de preguntas y respuestas.
4. Aplicar padding a las secuencias para asegurar longitudes consistentes dentro de cada batch.

A continuación, se muestra el código para realizar estos pasos de preprocesamiento.


In [7]:

#Direccion de contenido 
file_path = ('./ai-medical-chatbot.csv')

#file_path = "./ai-medical-chatbot.csv"
conversations_df = pd.read_csv(file_path)
# Limitar el dataframe a solo 40 filas
conversations_df = conversations_df.head(500)

print(conversations_df.head(20))


                                          Description  \
0       Q. What does abutment of the nerve root mean?   
1   Q. What should I do to reduce my weight gained...   
2   Q. I have started to get lots of acne on my fa...   
3   Q. Why do I have uncomfortable feeling between...   
4   Q. My symptoms after intercourse threatns me e...   
5   Q. I had a surgery which ended up with some fa...   
6                 Q. Kindly explain about Beta Strep.   
7   Q. Kindly suggest a homeopathic medicine to st...   
8       Q. What does abutment of the nerve root mean?   
9   Q. Will Nano-Leo give permanent solution for e...   
10  Q. Every time I eat spicy food, I poop blood. ...   
11  Q. Will Kalarchikai cure multiple ovarian cyst...   
12  Q. I masturbate only by rubbing the tip of the...   
13  Q. I have stomach pain and bloating after taki...   
14  Q. Can a painless tooth be extracted since it ...   
15  Q. What does the darker pigmentation at the ar...   
16  Q. I get fever during rainy

In [8]:

# Preparar las conversaciones
conversations = list(zip(conversations_df['Description'], conversations_df['Doctor']))

# Definir la clase para el dataset
class MedicalChatDataset(Dataset):
    def __init__(self, conversations, tokenizer):
        self.conversations = conversations
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.conversations)
    
    def __getitem__(self, idx):
        question, answer = self.conversations[idx]
        question_tokens = self.tokenizer(question)
        answer_tokens = self.tokenizer(answer)
        return torch.tensor(question_tokens), torch.tensor(answer_tokens)

# Función para hacer padding a las secuencias
def pad_sequences(batch):
    # Obtener las longitudes de todas las secuencias en el batch
    question_lengths = [len(x[0]) for x in batch]
    answer_lengths = [len(x[1]) for x in batch]
    
    # Obtener la longitud máxima para padding
    max_question_len = max(question_lengths)
    max_answer_len = max(answer_lengths)
    
    # Hacer padding en las preguntas y respuestas
    padded_questions = [torch.cat([q, torch.zeros(max_question_len - len(q))]) for q, _ in batch]
    padded_answers = [torch.cat([a, torch.zeros(max_answer_len - len(a))]) for _, a in batch]
    
    # Convertir a tensores
    padded_questions = torch.stack(padded_questions)
    padded_answers = torch.stack(padded_answers)
    
    return padded_questions.long(), padded_answers.long()

# Definir la arquitectura de la red LSTM
class MedicalChatbot(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers):
        super(MedicalChatbot, self).__init__()
        
        # Capa de embedding para convertir palabras en vectores
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        
        # Capa totalmente conectada para generar la salida
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out)  # Generar una salida para cada paso de la secuencia
        return output

# Función de entrenamiento
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        print(f"Iniciando época {epoch+1}/{num_epochs}")
        running_loss = 0.0
        for i, (inputs, targets) in enumerate(dataloader, 1):
            #print(f"Procesando el batch de datos {i}/{len(dataloader)}")
            optimizer.zero_grad()
            # Pasar las entradas por el modelo
            outputs = model(inputs)
            
            # Ajustar los targets para todas las palabras de la secuencia
            outputs = outputs.view(-1, outputs.size(-1))
            targets = targets.view(-1)
            
            # Asegurarse de que los tamaños coincidan
            if outputs.size(0) != targets.size(0):
                min_size = min(outputs.size(0), targets.size(0))
                outputs = outputs[:min_size]
                targets = targets[:min_size]
            
            # Calcular la pérdida
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Época [{epoch+1}/{num_epochs}] finalizada. Pérdida: {running_loss/len(dataloader)}')

# Definir el tokenizador mejorado con control de rangos
def simple_tokenizer(text, vocab_size=128):
    tokenized = []
    for c in text:
        if ord(c) < vocab_size:
            tokenized.append(ord(c))  # Usar el valor ASCII
        else:
            tokenized.append(0)  # Asignar un índice de "<UNK>" para caracteres fuera del rango
    return tokenized

# Parámetros del modelo
vocab_size = 128  # Mantener el vocabulario de caracteres ASCII básicos
embedding_dim = 64
hidden_dim = 128
output_dim = vocab_size  # Para generar una secuencia de caracteres ASCII
num_layers = 2
num_epochs = 1000
batch_size = 2

# Crear dataset y dataloader con collate_fn para padding
tokenizer = lambda text: simple_tokenizer(text, vocab_size)
dataset = MedicalChatDataset(conversations, tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_sequences)

# Crear el modelo
model = MedicalChatbot(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, num_epochs)




Iniciando época 1/1000
Época [1/1000] finalizada. Pérdida: 3.227552689552307
Iniciando época 2/1000
Época [2/1000] finalizada. Pérdida: 3.060571361541748
Iniciando época 3/1000
Época [3/1000] finalizada. Pérdida: 3.0407941370010376
Iniciando época 4/1000
Época [4/1000] finalizada. Pérdida: 3.0226345615386965
Iniciando época 5/1000
Época [5/1000] finalizada. Pérdida: 2.9856296043395996
Iniciando época 6/1000
Época [6/1000] finalizada. Pérdida: 2.997647988319397
Iniciando época 7/1000
Época [7/1000] finalizada. Pérdida: 2.963778060913086
Iniciando época 8/1000
Época [8/1000] finalizada. Pérdida: 2.9187767329216006
Iniciando época 9/1000
Época [9/1000] finalizada. Pérdida: 2.898981605052948
Iniciando época 10/1000
Época [10/1000] finalizada. Pérdida: 2.8718567757606506
Iniciando época 11/1000
Época [11/1000] finalizada. Pérdida: 2.867069733142853
Iniciando época 12/1000
Época [12/1000] finalizada. Pérdida: 2.7854934158325197
Iniciando época 13/1000
Época [13/1000] finalizada. Pérdida: 2.7

In [5]:

# Ejemplo de uso del chatbot
# Función para predecir la respuesta
def predict_response(model, question, tokenizer, vocab_size):
    model.eval()
    with torch.no_grad():
        # Tokenizar la pregunta y convertirla a tensor
        question_tokens = tokenizer(question)
        question_tensor = torch.tensor(question_tokens).unsqueeze(0)
        
        # Pasar la pregunta por el modelo
        output = model(question_tensor)
        
        # Obtener los índices de los tokens con la mayor probabilidad para cada paso de la secuencia
        predicted_token_idxs = torch.argmax(output, dim=2).squeeze().tolist()
        
        # Convertir los índices de vuelta a caracteres ASCII
        predicted_chars = [chr(idx) if idx < vocab_size else "<UNK>" for idx in predicted_token_idxs]
        
        return ''.join(predicted_chars)

user_input = "Hola, Tengo fiebre, Que puedo hacer?"

response = predict_response(model, user_input, tokenizer, vocab_size)
print(f"Chatbot: {response}")

Chatbot: loale  a   ne   o t ti    r  on  a t
